In [ ]:
from datetime import datetime
from dateutil import tz
import jquantsapi
from dotenv import load_dotenv
import os

load_dotenv()
my_mail_address:str = os.environ['JQUANTS_EMAIL']
my_password: str = os.environ['JQUANTS_PASSWORD']
cli = jquantsapi.Client(mail_address=my_mail_address, password=my_password)

## JQuantsの使用例

In [ ]:
# 上場銘柄一覧 /listed/info
#   MarketCode: 市場区分コード   
#   Sector17Code: 17業種コード
#   Sector33Code: 33業種コード

df = cli.get_listed_info()
df

In [ ]:
# 株価四本値 /prices/daily_quotes
#   code: 銘柄コード
#   from/to/date: 期間または日付

df = cli.get_prices_daily_quotes(code='8316')
df

In [ ]:
# 財務情報 /fins/statements
#   code: 銘柄コード
#   date: 日付

df = cli.get_fins_statements(code='8316')
df.to_excel('fins_statements_8316.xlsx')
df

## 必要なデータのダウンロード

In [ ]:
listed_info = cli.get_listed_info()

In [ ]:
import json
import os
from tqdm import tqdm
import pandas as pd

output_dir = 'jquants_data'
os.makedirs(output_dir, exist_ok=True)

class DateTimeEncoder(json.JSONEncoder):
    """
    pd.TimestampとdatetimeオブジェクトをISOフォーマットに変換するカスタムエンコーダー
    """
    def default(self, obj):
        if pd.isna(obj):
            return None
        elif isinstance(obj, (pd.Timestamp, datetime)):
            return obj.strftime('%Y-%m-%d')
        return super().default(obj)

for code in tqdm(listed_info['Code']):
    data = {
        'code': code,
        'prices_daily_quotes': cli.get_prices_daily_quotes(code=code).to_dict(orient='records'),
        'fins_statements': cli.get_fins_statements(code=code).to_dict(orient='records')
    }
    with open(os.path.join(output_dir, f'{code}.json'), 'w') as f:
        json.dump(data, f, cls=DateTimeEncoder, ensure_ascii=False, indent=2)